### Load new and old .sql dump

In [1]:
old = '/Users/Lu/dumps/Dump20160330.sql'
new = '/Users/Lu/dumps/Dump20160330.sql'

### Load packages

In [2]:
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

### Load sql magic

In [4]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Connect to local server

In [5]:
%sql mysql://root:@localhost

u'Connected: root@None'

### Create empty database first

In [7]:
%sql CREATE DATABASE IF NOT EXISTS softwaredata # need to create database first before loading .sql backup

1 rows affected.


[]

### Load sql dump file to create database

In [ ]:
#load .sql from command line wrong attemps:
#%sql mysqldump softwaredata >/Users/Lu/dumps/Dump20160330.sql
#%sql /Users/Lu/dumps/Dump20160330.sql
#%sql softwaredata < /Users/Lu/dumps/Dump20160330.sql
#%sql mysql -u root softwaredata </Users/Lu/dumps/Dump20160330.sql
#%sql source /Users/Lu/dumps/Dump20160330_copy.sql

In [8]:
#http://stackoverflow.com/questions/4408714/execute-sql-file-with-python-mysqldb
from subprocess import Popen, PIPE
process = Popen(['mysql', 'softwaredata', '-u', 'root'],
                stdout=PIPE, stdin=PIPE)
output=process.communicate('source ' + old)

### Create empty data base, rename imagescollection folder to a different name and save to the database ---  patient data from previous collection

In [9]:
%sql create database IF NOT EXISTS Images

1 rows affected.


/Users/Lu/anaconda/lib/python2.7/site-packages/pymysql/cursors.py:158: Warning: Can't create database 'Images'; database exists
  result = self._query(query)


[]

In [10]:
%sql DROP TABLE IF EXISTS Images.old_img
%sql CREATE TABLE Images.old_img SELECT IMCOLLID, SERID, ImageCollFolderName FROM softwaredata.imagescollection

0 rows affected.
181 rows affected.


[]

### Load new .sql backup file

In [11]:
#http://stackoverflow.com/questions/4408714/execute-sql-file-with-python-mysqldb
from subprocess import Popen, PIPE
process = Popen(['mysql', 'softwaredata', '-u', 'root'],
                stdout=PIPE, stdin=PIPE)
output=process.communicate('source ' + new)

In [12]:
%sql DROP TABLE IF EXISTS Images.new_img
%sql CREATE TABLE Images.new_img SELECT IMCOLLID, SERID, ImageCollFolderName FROM softwaredata.imagescollection

0 rows affected.
181 rows affected.


[]

### Compare two tables and extract list of new files

In [13]:
%sql use Images

0 rows affected.


[]

In [ ]:
# commandline: sudo rm /tmp/oldImgColl.csv 
#%sql select new_img.ImageCollFolderName from new_img left join old_img on new_img.ImageCollFolderName = old_img.ImageCollFolderName where new_img.ImageCollFolderName is NULL INTO OUTFILE '/tmp/oldImgColl.csv' 

In [14]:
%sql CREATE TABLE diff_img select new_img.ImageCollFolderName from new_img left join old_img on new_img.ImageCollFolderName = old_img.ImageCollFolderName where new_img.ImageCollFolderName is NULL  
# commandline: echo "select * from diff_img" | mysql --user=root Images > ~/Ipython/images.csv

0 rows affected.


[]